In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
cd drive/MyDrive/kaggle_competitions/house_prices


[Errno 2] No such file or directory: 'drive/MyDrive/kaggle_competitions/house_prices'
/content/drive/MyDrive/kaggle_competitions/house_prices


In [ ]:
pip install optuna


In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_log_error
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [164]:
data=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
ss=pd.read_csv('sample_submission.csv')

In [165]:
data.shape

(1460, 81)

In [166]:
targets=list(data.columns)
targets.remove('Id')
targets.remove('SalePrice')

In [167]:
data.set_index('Id')

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [168]:
Y=pd.DataFrame(data['SalePrice'])
X=pd.DataFrame(data)
#X=pd.concat([X,test])

In [169]:
missing_value_counts= (X.isnull().sum())
j=0
for i in missing_value_counts:
  if i>10:
    print(f'feature: {targets[j]}    misssing {i}')
  j=j+1
#missing_value_counts

feature: LotArea    misssing 259
feature: LotShape    misssing 1369
feature: BsmtCond    misssing 37
feature: BsmtExposure    misssing 37
feature: BsmtFinType1    misssing 38
feature: BsmtFinSF1    misssing 37
feature: BsmtFinSF2    misssing 38
feature: GarageType    misssing 690
feature: GarageYrBlt    misssing 81
feature: GarageFinish    misssing 81
feature: GarageCars    misssing 81
feature: GarageCond    misssing 81
feature: PavedDrive    misssing 81
feature: Fence    misssing 1453
feature: MiscFeature    misssing 1179
feature: MiscVal    misssing 1406


In [170]:
to_remove=['Alley','FireplaceQu','MiscFeature','PoolQC','Fence','YrSold','Utilities']

In [171]:
targetss=[targets]
for tar in targets:
  targetss.append(tar)



In [172]:

def preprocess(X,drop,to_remove=to_remove,targets=targets):

  X=X.drop(columns=to_remove)
  categorical_columns = X.select_dtypes(include=['object']).columns

# Convert categorical columns to numerical using one-hot encoding

  if drop:
    X=X.dropna()
  else:
    for col in targets:
      if col in categorical_columns:
        most_frequent_value = X[col].mode()[0]
        X[col] = X[col].fillna(most_frequent_value)
      else:
        X[col]=X[col].fillna(X[col].mean())



  return X


def get_dum(X):
  categorical_columns = X.select_dtypes(include=['object']).columns
  X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)
  return X

In [173]:
for x in to_remove:
    targets.remove(x)

In [174]:
mask = X['GarageType'].isna()

# Use the mask to get the rows where the condition is True
result = X[mask]
result.index

Int64Index([  39,   48,   78,   88,   89,   99,  108,  125,  127,  140,  148,
             155,  163,  165,  198,  210,  241,  250,  287,  291,  307,  375,
             386,  393,  431,  434,  441,  464,  495,  520,  528,  533,  535,
             562,  582,  613,  614,  620,  635,  636,  638,  649,  705,  710,
             738,  750,  784,  826,  843,  921,  942,  954,  960,  968,  970,
             976, 1009, 1011, 1030, 1038, 1096, 1123, 1131, 1137, 1143, 1173,
            1179, 1218, 1219, 1234, 1257, 1283, 1323, 1325, 1326, 1337, 1349,
            1407, 1449, 1450, 1453],
           dtype='int64')

In [175]:
mask13= X['Fireplaces'].isna()

# Use the mask to get the rows where the condition is True
result3 = X[mask]
result3.index

Int64Index([  39,   48,   78,   88,   89,   99,  108,  125,  127,  140,  148,
             155,  163,  165,  198,  210,  241,  250,  287,  291,  307,  375,
             386,  393,  431,  434,  441,  464,  495,  520,  528,  533,  535,
             562,  582,  613,  614,  620,  635,  636,  638,  649,  705,  710,
             738,  750,  784,  826,  843,  921,  942,  954,  960,  968,  970,
             976, 1009, 1011, 1030, 1038, 1096, 1123, 1131, 1137, 1143, 1173,
            1179, 1218, 1219, 1234, 1257, 1283, 1323, 1325, 1326, 1337, 1349,
            1407, 1449, 1450, 1453],
           dtype='int64')

# we can see that these lines have missing values in all these columns
feature: Fireplaces    misssing 81
feature: FireplaceQu    misssing 81
feature: GarageType    misssing 81
feature: GarageCars    misssing 81
feature: GarageArea    misssing 81

In [176]:
'''for target in targets:
    plt.subplots( figsize=(15, 3))
    sns.barplot(x=X[target], y=X['SalePrice'])
    plt.title(f" {target} vs price")
    plt.show()

for target in targets:
    plt.subplots( figsize=(15, 3))
    sns.barplot(x=X.index, y=X[target])
    plt.title(f" {target} ")
    plt.show()'''

'for target in targets:\n    plt.subplots( figsize=(15, 3))\n    sns.barplot(x=X[target], y=X[\'SalePrice\'])\n    plt.title(f" {target} vs price")\n    plt.show()\n\nfor target in targets:\n    plt.subplots( figsize=(15, 3))\n    sns.barplot(x=X.index, y=X[target])\n    plt.title(f" {target} ")\n    plt.show()'

In [177]:
'''import optuna
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)

    return rmse
study = optuna.create_study(direction='minimize')  # 'minimize' for minimizing the objective function
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and corresponding objective value
print('Best trial:')
trial = study.best_trial
print(f'Value: {trial.value}')
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')
xgb_params=study.best_params'''


"import optuna\ndef objective(trial):\n    params = {\n        'objective': 'reg:squarederror',\n        'eval_metric': 'rmse',\n        'booster': 'gbtree',\n        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),\n        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),\n        'max_depth': trial.suggest_int('max_depth', 1, 10),\n        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),\n        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),\n        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),\n        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),\n        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),\n        'n_estimators': trial.suggest_int('n_estimators', 50, 500),\n    }\n\n    model = xgb.XGBRegressor(**params)\n    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)\n\n    predictions = model.predict(X_val)\n    rmse = mean_squared_error(y_val, pre

In [231]:
# model parameters found by optuna
RANDOM_STATE=42


xgb_params = {'lambda': 3.903087410430548e-05,
 'alpha': 0.006143587510129687,
 'max_depth': 5,
 'eta': 0.6818356365090761,
 'gamma': 3.210131091050978e-08,
 'colsample_bytree': 0.953415283176276,
 'subsample': 0.8875510953635628,
 'min_child_weight': 3,
 'n_estimators': 331}

# Train AND validation

In [229]:
X1=pd.DataFrame(X)
X1=preprocess(X1,True,to_remove,targets)
Y1=pd.DataFrame(X1['SalePrice'])
X1=get_dum(X1)

In [232]:
import xgboost as xgb

X_train,X_val,y_train,y_val=train_test_split(X1,Y1,test_size=0.2)


# Create an XGBoost model
xgb_model = xgb.XGBRegressor(**xgb_params)

# Fit the model
xgb_model.fit(X_train, y_train)
#prediction
predictions = xgb_model.predict(X_val)

# Evaluate the model
mse = mean_squared_log_error(y_val, predictions)
print(f'mean_squared_log_error: {mse}')

mean_squared_log_error: 0.0010532017972430469


#Real Submission

In [224]:
#Code to submit
X2=pd.DataFrame(X)
X2=preprocess(X2,True,to_remove,targets)
test=preprocess(test,False,to_remove,targets)
merged=pd.concat([X2,test])
merged=get_dum(merged)
Y2=merged['SalePrice'].iloc[: 1094]
X2=merged.drop(columns=['SalePrice']).iloc[: 1094]
X_test_data=merged.drop(columns=['SalePrice']).iloc[1094 :]


# Create an XGBoost model
xgb_model = xgb.XGBRegressor(**xgb_params)

# Fit the model
xgb_model.fit(X2, Y2)

# Make predictions
predictions = xgb_model.predict(X_test_data)
predictions=pd.DataFrame(predictions)
predictions.columns=['SalePrice']
ss['SalePrice']=predictions['SalePrice']
ss=ss.set_index('Id')
ss.to_csv('my_submission__.csv')

,SalePrice
0,115693.984375
1,168613.437500
2,190094.390625
3,187665.250000
4,213749.703125
...,...
1454,98780.367188
1455,91656.710938
1456,196478.156250
1457,130630.437500


Index(['SalePrice'], dtype='object')